In [1]:
from tensorflow import keras
import numpy as np
from matplotlib import pyplot as plt
import cv2
from pathlib import Path

In [2]:
paths = ["train/cats", "train/dogs"] 
imgs = []  # Lista para almacenar las imágenes
labels = []  # Lista para almacenar las etiquetas

for idx, path in enumerate(paths):
    path_images = Path(path).glob("**/*.jpg") 
    path_images = [str(x) for x in path_images]
    
    for path_image in path_images:
        img = cv2.imread(path_image, 1)  # Leer la imagen en modo de escala de grises
        img = img / 255  # Normalizar la imagen
        imgs.append(cv2.resize(img, (200, 200), cv2.INTER_AREA))  # Redimensionar y añadir a la lista
    
    labels.extend([idx] * len(path_images))  # Añadir etiquetas correspondientes

# Convertir las listas a arreglos de NumPy después del bucle
imgs = np.array(imgs)
labels = np.array(labels)

print(f"Images shape: {imgs.shape}")
print(f"Num. of labels: {len(labels)}")

Images shape: (557, 200, 200, 3)
Num. of labels: 557


In [4]:
# Crear un modelo secuencial correctamente
model = keras.Sequential([
    keras.layers.Input((200, 200, 3)), # Los dos primeros don las dimensiones, el último es la escala de color
    keras.layers.Conv2D(32,3, padding="same", strides=1, activation="relu"), 
    keras.layers.MaxPooling2D(), 
    keras.layers.Conv2D(64,3, padding="same", strides=1),
    keras.layers.MaxPooling2D(),
    keras.layers.Conv2D(128,3, padding="same", strides=1),
    keras.layers.MaxPooling2D(),
    keras.layers.Conv2D(256,3, padding="same", strides=1),
    keras.layers.MaxPooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(512,activation="relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(128),
    keras.layers.Dense(1, activation="sigmoid")
])

# Imprimir el resumen del modelo
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 200, 200, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 100, 100, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 100, 100, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 50, 50, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 50, 50, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 25, 25, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 25, 25, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 12, 12, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 36864)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │    18,874,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,329,089 (73.73 MB)

 Trainable params: 19,329,089 (73.73 MB)

 Non-trainable params: 0 (0.00 B)

None


In [7]:
model.compile(loss=keras.losses.BinaryCrossentropy(), optimizer=keras.optimizers.Adam(learning_rate=0.0001), metrics=["accuracy"])
history = model.fit(imgs, labels, epochs=20, batch_size=8, validation_split = 0.1)

Epoch 1/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 16s 224ms/step - accuracy: 0.4753 - loss: 0.7726 - val_accuracy: 0.0000e+00 - val_loss: 0.9009
Epoch 2/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 12s 188ms/step - accuracy: 0.6288 - loss: 0.6749 - val_accuracy: 0.8750 - val_loss: 0.6192
Epoch 3/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 12s 189ms/step - accuracy: 0.6696 - loss: 0.6394 - val_accuracy: 0.6607 - val_loss: 0.6413
Epoch 4/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 12s 188ms/step - accuracy: 0.7346 - loss: 0.5572 - val_accuracy: 0.5714 - val_loss: 0.6848
Epoch 5/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 12s 188ms/step - accuracy: 0.7636 - loss: 0.4966 - val_accuracy: 0.3750 - val_loss: 1.1571
Epoch 6/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 12s 187ms/step - accuracy: 0.8320 - loss: 0.3980 - val_accuracy: 0.6429 - val_loss: 0.5634
Epoch 7/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 12s 188ms/step - accuracy: 0.8726 - loss: 0.2987 - val_accuracy: 0.5893 - val_loss: 0.7117
Epoch 8/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 12s 188ms/step - accuracy: 0.9393 - loss: 0.2031 - val_

In [10]:
paths = ["test/cats", "test/dogs"]  # Uso de barras normales
imgs_test = []  # Lista para almacenar las imágenes
labels_test = []  # Lista para almacenar las etiquetas

for idx, path in enumerate(paths):
    path_images = Path(path).glob("**/*.jpg")
    path_images = [str(x) for x in path_images]
    
    for path_image in path_images:
        img = cv2.imread(path_image, 1)  # Leer la imagen en modo de escala de grises
        img = img / 255  # Normalizar la imagen
        imgs_test.append(cv2.resize(img, (200, 200), cv2.INTER_AREA))  # Redimensionar y añadir a la lista
    
    labels_test.extend([idx] * len(path_images))  # Añadir etiquetas correspondientes

# Convertir las listas a arreglos de NumPy después del bucle
imgs_test = np.array(imgs_test)
labels_test = np.array(labels_test)

print(f"Images shape: {imgs_test.shape}")
print(f"Num. of labels_test: {len(labels_test)}")

Images shape: (140, 200, 200, 3)
Num. of labels_test: 140


In [11]:
predictions = model.predict(imgs_test)
predictions = np.argmax(predictions, axis = 1)
print(((predictions == labels_test).sum()/len(labels_test))*100)

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 117ms/step
50.0
